# Detect Model Bias with Amazon SageMaker Clarify

# Terminology

* **Bias**: 
An imbalance in the training data or the prediction behavior of the model across different groups, such as age or income bracket. Biases can result from the data or algorithm used to train your model. For instance, if an ML model is trained primarily on data from middle-aged individuals, it may be less accurate when making predictions involving younger and older people.

* **Bias metric**: 
A function that returns numerical values indicating the level of a potential bias.

* **Bias report**:
A collection of bias metrics for a given dataset, or a combination of a dataset and a model.

* **Label**:
Feature that is the target for training a machine learning model. Referred to as the observed label or observed outcome.

* **Positive label values**:
Label values that are favorable to a demographic group observed in a sample. In other words, designates a sample as having a positive result.

* **Negative label values**:
Label values that are unfavorable to a demographic group observed in a sample. In other words, designates a sample as having a negative result.

* **Facet**:
A column or feature that contains the attributes with respect to which bias is measured.

* **Facet value**:
The feature values of attributes that bias might favor or disfavor.

# Posttraining Bias Metrics
https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html

* **Difference in Positive Proportions in Predicted Labels (DPPL)**:
Measures the difference in the proportion of positive predictions between the favored facet a and the disfavored facet d.

* **Disparate Impact (DI)**:
Measures the ratio of proportions of the predicted labels for the favored facet a and the disfavored facet d.

* **Difference in Conditional Acceptance (DCAcc)**:
Compares the observed labels to the labels predicted by a model and assesses whether this is the same across facets for predicted positive outcomes (acceptances).

* **Difference in Conditional Rejection (DCR)**:
Compares the observed labels to the labels predicted by a model and assesses whether this is the same across facets for negative outcomes (rejections).

* **Recall Difference (RD)**:
Compares the recall of the model for the favored and disfavored facets.

* **Difference in Acceptance Rates (DAR)**:
Measures the difference in the ratios of the observed positive outcomes (TP) to the predicted positives (TP + FP) between the favored and disfavored facets.

* **Difference in Rejection Rates (DRR)**:
Measures the difference in the ratios of the observed negative outcomes (TN) to the predicted negatives (TN + FN) between the disfavored and favored facets.

* **Accuracy Difference (AD)**:
Measures the difference between the prediction accuracy for the favored and disfavored facets.

* **Treatment Equality (TE)**:
Measures the difference in the ratio of false positives to false negatives between the favored and disfavored facets.

* **Conditional Demographic Disparity in Predicted Labels (CDDPL)**:
Measures the disparity of predicted labels between the facets as a whole, but also by subgroups.

* **Counterfactual Fliptest (FT)**:
Examines each member of facet d and assesses whether similar members of facet a have different model predictions.


In [2]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
bucket

'sagemaker-us-east-1-117859797117'

In [3]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Test data for bias

We created test data in JSONLines format to match the model inputs. 

In [4]:
test_data_bias_path = "./data-clarify/test_data_bias.jsonl"

In [5]:
!head -n 1 $test_data_bias_path

{"features":["I have been using Quicken for years now and it does everything that I need it to accomplish for my personal finances.","Digital_Software"],"star_rating":4}


### Upload the data

In [6]:
test_data_bias_s3_uri = sess.upload_data(bucket=bucket, key_prefix="bias/test_data_bias", path=test_data_bias_path)
test_data_bias_s3_uri

's3://sagemaker-us-east-1-117859797117/bias/test_data_bias/test_data_bias.jsonl'

In [7]:
!aws s3 ls $test_data_bias_s3_uri

2021-04-14 13:41:19     197983 test_data_bias.jsonl


In [8]:
%store test_data_bias_s3_uri

Stored 'test_data_bias_s3_uri' (str)


# Run Posttraining Model Bias Analysis

In [9]:
%store -r training_job_name

In [10]:
print(training_job_name)

tensorflow-training-2021-04-14-10-42-44-832


## Create Model

In [11]:
import sagemaker

inference_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.3.1",
    py_version="py37",
    instance_type="ml.m5.4xlarge",
    image_scope="inference",
)
print(inference_image_uri)

763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.1-cpu


In [12]:
model_name = sess.create_model_from_job(training_job_name=training_job_name, image_uri=inference_image_uri)
print(model_name)

tensorflow-training-2021-04-14-10-42-44-832


# SageMakerClarifyProcessor

In [13]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.c5.2xlarge", sagemaker_session=sess
)

# Writing DataConfig and ModelConfig
A `DataConfig` object communicates some basic information about data I/O to Clarify. We specify where to find the input dataset, where to store the output, the target column (`label`), the header names, and the dataset type.

Similarly, the `ModelConfig` object communicates information about your trained model and `ModelPredictedLabelConfig` provides information on the format of your predictions.  

**Note**: To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing. `ModelConfig` specifies your preferred instance type and instance count used to run your model on during Clarify's processing.

## DataConfig

In [14]:
bias_report_prefix = "bias/report-{}".format(training_job_name)

bias_report_output_path = "s3://{}/{}".format(bucket, bias_report_prefix)

data_config = clarify.DataConfig(
    s3_data_input_path=test_data_bias_s3_uri,
    s3_output_path=bias_report_output_path,
    label="star_rating",
    features="features",
    # label must be last, features in exact order as passed into model
    headers=["review_body", "product_category", "star_rating"],
    dataset_type="application/jsonlines",
)

## ModelConfig

In [15]:
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.4xlarge",
    instance_count=1,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    # {"features": ["the worst", "Digital_Software"]}
    content_template='{"features":$features}',
)

## _Note: `label` is set to the JSON key for the model prediction results_

In [16]:
predictions_config = clarify.ModelPredictedLabelConfig(label="predicted_label")

## BiasConfig

In [17]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[
        5,
        4,
    ],  # needs to be int or str for continuous dtype, needs to be >1 for categorical dtype
    facet_name="product_category",
    #                                facet_values_or_threshold=['Gift Card'],
    group_name="product_category",
)

# Run Clarify Job

In [18]:
clarify_processor.run_post_training_bias(
    data_config=data_config,
    data_bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    #    methods='all', # FlipTest requires all columns to be numeric
    methods=["DPPL", "DI", "DCA", "DCR", "RD", "DAR", "DRR", "AD", "CDDPL", "TE"],
    wait=False,
    logs=False,
)


Job Name:  Clarify-Posttraining-Bias-2021-04-14-13-41-24-410
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-117859797117/bias/test_data_bias/test_data_bias.jsonl', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-117859797117/bias/report-tensorflow-training-2021-04-14-10-42-44-832/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-117859797117/bias/report-tensorflow-training-2021-04-14-10-42-44-832', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode':

In [19]:
run_post_training_bias_processing_job_name = clarify_processor.latest_job.job_name
run_post_training_bias_processing_job_name

'Clarify-Posttraining-Bias-2021-04-14-13-41-24-410'

In [20]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [21]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [22]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, bias_report_prefix
        )
    )
)

In [23]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=run_post_training_bias_processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

{'ProcessingInputs': [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-117859797117/bias/test_data_bias/test_data_bias.jsonl', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-117859797117/bias/report-tensorflow-training-2021-04-14-10-42-44-832/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'analysis_result', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-117859797117/bias/report-tensorflow-training-2021-04-14-10-42-44-832', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}, 'AppManaged': False}]}, 'Pro

In [24]:
running_processor.wait(logs=False)

......................................................................................................................................................!

# Download Report From S3

In [25]:
!aws s3 ls $bias_report_output_path/

2021-04-14 13:53:53       7699 analysis.json
2021-04-14 13:42:10        750 analysis_config.json
2021-04-14 13:53:53     277769 report.html
2021-04-14 13:53:53       6584 report.ipynb
2021-04-14 13:53:53      41552 report.pdf


In [26]:
!aws s3 cp --recursive $bias_report_output_path ./generated_bias_report/

download: s3://sagemaker-us-east-1-117859797117/bias/report-tensorflow-training-2021-04-14-10-42-44-832/analysis.json to generated_bias_report/analysis.json
download: s3://sagemaker-us-east-1-117859797117/bias/report-tensorflow-training-2021-04-14-10-42-44-832/analysis_config.json to generated_bias_report/analysis_config.json
download: s3://sagemaker-us-east-1-117859797117/bias/report-tensorflow-training-2021-04-14-10-42-44-832/report.pdf to generated_bias_report/report.pdf
download: s3://sagemaker-us-east-1-117859797117/bias/report-tensorflow-training-2021-04-14-10-42-44-832/report.ipynb to generated_bias_report/report.ipynb
download: s3://sagemaker-us-east-1-117859797117/bias/report-tensorflow-training-2021-04-14-10-42-44-832/report.html to generated_bias_report/report.html


In [27]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./generated_bias_report/report.html">Bias Report</a></b>'))